## 作业二

### 数据预处理

In [7]:
import pandas as pd
import numpy as np

file_path = 'data/netflix_train.txt'
data_train = pd.read_csv(file_path, sep=r'\s+', header=None, names=['user_id', 'movie_id', 'rating', 'timestamp'])
print(data_train.head())

users_train = data_train['user_id'].unique()
movies_train = data_train['movie_id'].unique()

X_train = pd.DataFrame(0, index=users_train, columns=movies_train)

for row in data_train.itertuples():
    X_train.at[row.user_id, row.movie_id] = row.rating

print("Matrix for Train:")
print(X_train)

# X.to_csv('user_movie_matrix.csv')
# missing_mask.to_csv('missing_mask.csv')


   user_id  movie_id  rating   timestamp
0   305344         1       1  2004-02-08
1   305344         3       2  2003-03-23
2   305344         4       1  2003-12-13
3   305344         5       1  2004-12-15
4   305344         6       1  2003-09-23
Matrix for Train:
         1     3     4     5     6     7     8     10    15    17    ...  \
305344      1     2     1     1     1     2     1     1     5     2  ...   
387418      1     2     0     1     2     1     0     0     0     3  ...   
2439493     0     0     1     0     1     0     1     1     0     1  ...   
1664010     5     0     3     5     4     3     4     4     4     0  ...   
2118461     5     4     3     5     4     3     0     3     0     3  ...   
...       ...   ...   ...   ...   ...   ...   ...   ...   ...   ...  ...   
1688557     0     0     0     0     0     0     0     0     0     0  ...   
959444      0     0     0     0     0     0     0     0     0     0  ...   
1581163     0     0     0     0     0     0     3   

In [12]:
file_path = 'data/netflix_test.txt'
data_test = pd.read_csv(file_path, sep=r'\s+', header=None, names=['user_id', 'movie_id', 'rating', 'timestamp'])
print(data_test.head())

users_test = data_test['user_id'].unique()
movies_test = data_test['movie_id'].unique()
print(len(data_test['movie_id'].unique())) #9983

X_test = pd.DataFrame(0, index=users_test, columns=movies_test)
print(X_test.shape)
for row in data_test.itertuples():
    X_test.at[row.user_id, row.movie_id] = row.rating

print("Matrix for Test:")
print(X_test)

   user_id  movie_id  rating   timestamp
0  2123534      6134       4  2005-05-31
1  2123534       677       1  2005-06-06
2  2123534      3182       4  2005-06-06
3  2123534      7882       4  2005-06-27
4  2123534      6099       1  2005-05-31
9983
(10000, 9983)
Matrix for Test:
         6134  677   3182  7882  6099  1295  6336  8764  8872  4883  ...  \
2123534     4     1     4     4     1     1     1     4     3     2  ...   
282427      0     0     0     0     0     3     4     0     0     0  ...   
2065448     0     0     0     0     0     0     0     0     0     0  ...   
1364974     0     0     0     0     0     0     0     0     0     0  ...   
1035378     0     0     0     3     0     0     0     5     0     4  ...   
...       ...   ...   ...   ...   ...   ...   ...   ...   ...   ...  ...   
1018928     0     0     0     0     0     0     0     0     0     0  ...   
1268171     0     0     0     0     2     0     0     0     0     2  ...   
492291      0     0     0     0   

### 协同过滤

In [13]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import time



计算用户相似度耗时: 2.3220 秒


In [19]:
# 计算用户间的余弦相似度
start_time = time.time()
user_similarity = cosine_similarity(X_train.fillna(0))  # 将 NaN 填充为 0
user_similarity_df = pd.DataFrame(user_similarity, index=X_train.index, columns=X_train.index)
end_time = time.time()

print(f"计算用户相似度耗时: {end_time - start_time:.4f} 秒")
# 预测评分函数
def predict_rating(user_id, movie_id):
    if movie_id not in X_train.columns:
        return 0  # 如果电影不在矩阵中，返回 0
    similar_users = user_similarity_df[user_id].drop(user_id)  # 获取与用户的相似度
    relevant_users = similar_users[similar_users > 0]  # 只考虑相似度大于 0 的用户

    # 计算加权评分
    weighted_sum = 0
    similarity_sum = 0
    for k in relevant_users.index:
        if X_train.at[k, movie_id] > 0:  # 只有已评分的电影
            weighted_sum += relevant_users[k] * X_train.at[k, movie_id]
            similarity_sum += relevant_users[k]

    if similarity_sum == 0:
        return 0  # 如果没有相似用户评分，返回 0
    return weighted_sum / similarity_sum

# 评估 RMSE
def calculate_rmse(test_data):
    total_error = 0
    count = 0
    for _, row in test_data.iterrows():
        user_id = row['user_id']
        movie_id = row['movie_id']
        actual_rating = row['rating']
        predicted_rating = predict_rating(user_id, movie_id)
        total_error += (actual_rating - predicted_rating) ** 2
        count += 1
    return np.sqrt(total_error / count)
rmse = calculate_rmse(X_test)
print(f"最终 RMSE 结果: {rmse:.4f}")

# 计算整个过程的时间
end_time_total = time.time()
print(f"总耗时: {end_time_total - start_time:.4f} 秒")

计算用户相似度耗时: 2.3340 秒


KeyError: 'user_id'